# Support Vector Machines (SVM) Tutorial

This notebook accompanies the SVM tutorial PDF. It demonstrates SVM using the Iris dataset, compares kernels (linear, polynomial, RBF), shows decision boundaries (using two features), performs hyperparameter search, and evaluates results.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib


In [ ]:
# Load Iris dataset and prepare a binary classification problem: Setosa vs Versicolor
iris = datasets.load_iris()
X = iris.data[:, [0, 2]]  # use sepal length (0) and petal length (2) for easy visualization
y = iris.target

# Select two classes: class 0 (setosa) and class 1 (versicolor)
mask = (y == 0) | (y == 1)
X = X[mask]
y = y[mask]
print('Dataset shape:', X.shape)
print('Class distribution:', np.bincount(y))


In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Helper to plot decision boundaries for 2D data
def plot_decision_boundaries(clf, X, y, title='Decision boundary'):
    # create grid
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300), np.linspace(y_min, y_max, 300))
    grid = np.c_[xx.ravel(), yy.ravel()]
    Z = clf.predict(grid)
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(8,6))
    plt.contourf(xx, yy, Z, alpha=0.2)
    plt.scatter(X[:,0], X[:,1], c=y, edgecolor='k')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title(title)
    plt.show()


In [ ]:
# Train SVMs with different kernels and show results
kernels = ['linear', 'poly', 'rbf']
models = {}
for k in kernels:
    if k == 'poly':
        clf = svm.SVC(kernel=k, degree=3, C=1.0, probability=True)
    else:
        clf = svm.SVC(kernel=k, C=1.0, probability=True)
    clf.fit(X_train_scaled, y_train)
    models[k] = clf
    y_pred = clf.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    print(f"Kernel={k:6s}  Test accuracy={acc:.4f}")
    plot_decision_boundaries(clf, np.vstack([X_train_scaled, X_test_scaled]), np.hstack([y_train, y_test]), title=f'SVM ({k})')


In [ ]:
# Grid search for RBF kernel hyperparameters (C and gamma)
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1], 'kernel': ['rbf']}
grid = GridSearchCV(svm.SVC(), param_grid, cv=5, scoring='accuracy')
grid.fit(X_train_scaled, y_train)
print('Best params (RBF):', grid.best_params_)
best_rbf = grid.best_estimator_
y_pred = best_rbf.predict(X_test_scaled)
print('Test acc (best RBF):', accuracy_score(y_test, y_pred))
plot_decision_boundaries(best_rbf, np.vstack([X_train_scaled, X_test_scaled]), np.hstack([y_train, y_test]), title='Best RBF (GridSearch)')


In [ ]:
# Confusion matrix and classification report for best model
print('Confusion matrix:')
print(confusion_matrix(y_test, y_pred))
print('\nClassification report:')
print(classification_report(y_test, y_pred))


In [ ]:
# Save model and scaler
joblib.dump(best_rbf, 'svm_rbf_best.joblib')
joblib.dump(scaler, 'scaler.joblib')
print('Saved model: svm_rbf_best.joblib and scaler.joblib')


## Notes

- The notebook uses only two features so decision boundaries can be visualized.
- For real work, consider using all features and cross-validation pipelines.
- Remember to cite sources in your report (e.g., Cortes & Vapnik 1995, scikit-learn docs).
